In [2]:
import logging
import time
from IPython.core.display import display
from sink.visualization_sink import PlotlyVisualizationSink
from source.mqtt_source import MQTTSource
from source.mqtt_wrapper import MqttClientWrapper
from plotly import graph_objects as go
import pandas as pd
import datetime
import numpy as np
from rx import scheduler
from utils.logging import configure_logger_to_output
logger_output = configure_logger_to_output(logging.root)
logging.root.setLevel(logging.INFO)
import rx
from rx import operators
if 'RUN_ONCE' not in globals():
    %cd ..
    RUN_ONCE = True
    

In [3]:
# Test update random plot
logger_output.clear_output()
plotly_sink = PlotlyVisualizationSink()

rx.interval(2.0).pipe(
    operators.map(lambda *args: np.random.random(200)),
    operators.map(lambda y: [{"type": "scatter", "y":y}]),
    operators.take_until_with_time(10)
).subscribe(plotly_sink)


display(logger_output)
display(plotly_sink.figure)




Output()

FigureWidget({
    'data': [], 'layout': {'template': '...', 'title': {'text': 'plot'}}
})

In [9]:
# Test update timeseries plot
logger_output.clear_output()
plotly_sink = PlotlyVisualizationSink()

def acc(x: pd.DataFrame, a):
    return x.append(pd.DataFrame([{"time": datetime.datetime.now(), "y": a, "x":a}]))

sc = scheduler.NewThreadScheduler()
rx.interval(1.0, sc).pipe(
    operators.scan(acc, pd.DataFrame()),
    operators.map(lambda x: x.tail(5)),
    operators.map(lambda x: [go.Scatter(y=x.y, x=x.x)]),
    operators.take_until_with_time(20),
).subscribe(plotly_sink)


display(logger_output)
display(plotly_sink.figure)



